In [1]:
import cv2
import mediapipe as mp
import pandas as pd
import os
from pathlib import Path
from collections import defaultdict
import numpy as np

# 관절 이름 매핑
LANDMARK_NAMES = [
    "nose", "left_eye_inner", "left_eye", "left_eye_outer", "right_eye_inner", "right_eye",
    "right_eye_outer", "left_ear", "right_ear", "mouth_left", "mouth_right", "left_shoulder",
    "right_shoulder", "left_elbow", "right_elbow", "left_wrist", "right_wrist", "left_pinky",
    "right_pinky", "left_index", "right_index", "left_thumb", "right_thumb", "left_hip",
    "right_hip", "left_knee", "right_knee", "left_ankle", "right_ankle", "left_heel",
    "right_heel", "left_foot_index", "right_foot_index"
]

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

def imread_unicode(path):
    try:
        stream = np.fromfile(str(path), dtype=np.uint8)
        img = cv2.imdecode(stream, cv2.IMREAD_COLOR)
        return img
    except Exception as e:
        print(f"[예외 발생] {path} → {e}")
        return None
    
def extract_pose(image_path):
    image = imread_unicode(image_path)
    if not image_path.exists():
        print(f"[파일 없음] {image_path}")
    if image is None:
        print(f"[로드 실패] {image_path}")
        return None

    with mp_pose.Pose(static_image_mode=True) as pose:
        results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        if not results.pose_landmarks:
            # 관절 미검출 시 모든 값을 NaN으로 처리
            return {name: {"x": np.nan, "y": np.nan, "z": np.nan, "visibility": np.nan} for name in LANDMARK_NAMES}

        landmarks = results.pose_landmarks.landmark
        data = {
            name: {
                "x": landmarks[idx].x if landmarks[idx].visibility > 0 else np.nan,
                "y": landmarks[idx].y if landmarks[idx].visibility > 0 else np.nan,
                "z": landmarks[idx].z if landmarks[idx].visibility > 0 else np.nan,
                "visibility": landmarks[idx].visibility if landmarks[idx].visibility > 0 else np.nan
            }
            if idx < len(landmarks) else {
                "x": np.nan, "y": np.nan, "z": np.nan, "visibility": np.nan
            }
            for idx, name in enumerate(LANDMARK_NAMES)
        }
        return data

def process_directory(input_dir, output_dir):
    input_dir = Path(input_dir)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    # 시퀀스별로 파일 그룹화
    sequences = defaultdict(list)
    for img_path in input_dir.glob("*.jpg"):
        parts = img_path.stem.split("_")
        if len(parts) < 6:
            print(f"[경고] 잘못된 파일명: {img_path.name}")
            continue
        sequence_name = "_".join(parts[:-1])
        sequences[sequence_name].append(img_path)


    # 각 시퀀스 처리
    for sequence, files in sequences.items():
        sequence_data = []
        files = sorted(files)  # 시퀀스 정렬
        for img_path in files:
            pose_data = extract_pose(img_path)
            if pose_data is None:
                continue
            pose_data_flat = {}
            for joint_name, values in pose_data.items():
                for key in ["x", "y", "z", "visibility"]:
                    pose_data_flat[f"{joint_name}_{key}"] = values[key]
            pose_data_flat["filename"] = img_path.name
            sequence_data.append(pose_data_flat)

        if sequence_data:
            df = pd.DataFrame(sequence_data)
            save_path = output_dir / f"{sequence}.csv"
            df.to_csv(save_path, index=False)
            print(f"[저장 완료] {save_path}")
        else:
            print(f"[스킵] 유효한 프레임 없음: {sequence}")

# ✅ 처리 대상 경로 설정
base_dir = Path(r"D:\golfDataset\스포츠 사람 동작 영상(골프)\Training\Public\male\tf")

process_directory(
    input_dir=base_dir / "true" / "cropped",
    output_dir=base_dir / "true" / "skel_csv"
)

process_directory(
    input_dir=base_dir / "false" / "cropped",
    output_dir=base_dir / "false" / "skel_csv"
)


[저장 완료] D:\golfDataset\스포츠 사람 동작 영상(골프)\Training\Public\male\tf\true\skel_csv\20201116_General_001_DOS_A_M40_MM_001.csv
[저장 완료] D:\golfDataset\스포츠 사람 동작 영상(골프)\Training\Public\male\tf\true\skel_csv\20201116_General_001_DOS_A_M40_MM_002.csv
[저장 완료] D:\golfDataset\스포츠 사람 동작 영상(골프)\Training\Public\male\tf\true\skel_csv\20201116_General_001_DOS_A_M40_MM_003.csv
[저장 완료] D:\golfDataset\스포츠 사람 동작 영상(골프)\Training\Public\male\tf\true\skel_csv\20201116_General_001_DOS_A_M40_MM_007.csv
[저장 완료] D:\golfDataset\스포츠 사람 동작 영상(골프)\Training\Public\male\tf\true\skel_csv\20201116_General_001_DOS_A_M40_MM_008.csv
[저장 완료] D:\golfDataset\스포츠 사람 동작 영상(골프)\Training\Public\male\tf\true\skel_csv\20201116_General_001_DOS_A_M40_MM_009.csv
[저장 완료] D:\golfDataset\스포츠 사람 동작 영상(골프)\Training\Public\male\tf\true\skel_csv\20201116_General_001_DOS_A_M40_MM_010.csv
[저장 완료] D:\golfDataset\스포츠 사람 동작 영상(골프)\Training\Public\male\tf\true\skel_csv\20201116_General_001_DOS_A_M40_MM_011.csv
[저장 완료] D:\golfDataset\스포츠 사람 동작 영상(골프)\

KeyboardInterrupt: 